<a href="https://colab.research.google.com/github/int29/PDLP/blob/main/chapter_01_Basic_Image_Classification_with_FFNN_on_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 내가쓰려고 만든 pytorch로 구현하는 신경망 부터 LLM 까지
* INT29
* Start_Date : 2024-01-24
* End_Date :

In [1]:
# python 버전
!python --version

Python 3.10.12


In [2]:
# torch, torchvision 버전

import torch, torchvision
print(torch.__version__)
print(torchvision.__version__)


2.1.0+cu121
0.16.0+cu121


In [3]:
# GPU 및 CUDA환경
!nvidia-smi

Sun Feb 11 04:07:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Chapter 01. FFNN을 활용한 기본적인 CIFAL10 이미지 분류모델

### 목표
* CIFAL10 데이터를 가장 기본적인 FFNN을 통해 분류하고 한다.

### Feed Forward Neural Network (F.F.N.N) 소개

순방향 인공신경망(Feed Forward Nueral Network)은 가장 기본이 되는 신경망으로 일반적으로 인공신경망(Aritificial Neural Network)이라고 하면 이 FFNN을 의미한다.

딥러닝은 인공신경망이 가장 최적화된 특징을 학습할 수 있도록 Auto Feature Extraction 패러다임을 갖고있다.[1] 따라서 최신 알고리즘이라고 불리는 CNN, RNN, LSTM, GAN, Transformer 또한 이미지, 텍스트의 특징을 잘 추출하는 알고리즘일 뿐 결국 최종적인 분류 및 예측작업은 항상 FFNN이 처리하게 된다. 따라서 정확하게 이해하고 넘어가야하는 중요한 알고리즘이다.


#### 01 Feed Forward Neural Network 정의

순방향 신경망(Feed Forward Neural Network)은 가장 기본적인 인공 신경망 구조이다.
신경망을 순방항(feedforward)이라 부르는 이유는 입력층에서 은닉층을 거쳐 출력층으로 역행하는 부분이 없이 정보가 한 방향으로만 흐르는기 때문이다. [2]

Feed Forward Neural Network에 대한 설명은 Github의 PDF에 자세하게 설명하였다.

#### 라이브러리 로드

In [4]:
#import librarys

# warnings off
import warnings
warnings.filterwarnings('ignore')

import torch
from torchvision import transforms
from torchvision import datasets

import torch.nn as nn
import torch.nn.functional as F

In [5]:
# check if gpu is available(Cuda GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

#### 02 Pytorch 모델 기본 플로우

Pytorch를 통해 모델을 개발하는 과정은 (1)데이터 로드 및 데이터 전처리, (2)모델아키텍쳐 생성, (3)훈련 및 검증 함수 정의, (4)훈련 및 테스트 정의 4가지 과정으로 이루어진다.

(1) 데이터 로드 및 데이터 전처리: Pytorch의 DataLoader와 transforms 모듈을 통해 데이터를 batch학습 및 전처리를 위한 파이프라인을 생성하고 처리하는 과정.
* (1-1)데이터 전처리 설정
* (1-2)데이터셋 및 DataLoader 준비

(2)모델아키텍쳐 생성: 실제 Pytorch 신경망을 구성하는 과정.

(3)훈련 및 검증 함수 정의:
* (3-1) train() 함수에서는 모델을 훈련 모드로 설정, 데이터 로더를 통해 데이터와 타깃을 네트워크에 전달하고, 손실을 계산한 후 역전파로 매개변수를 업데이트.
* (3-2) test() 함수에서는 모델을 평가 모드로 설정, 기울기 계산을 비활성화한 상태에서 데이터 로더를 통해 데이터를 네트워크에 전달하고, 손실 및 정확도를 계산.

(4)훈련 및 테스트: (1)~(3)까지 정의한 결과를 토대로 학습을 진행해 파라미터를 최적화하고 결과를 스코어링.
* (4-1)손실 함수 및 최적화 알고리즘 정의
* (4-2)설정한 에폭 수만큼 반복하여 각 에폭마다 훈련 및 검증(테스트) 실행.

#### (1) 데이터 로드 및 데이터 전처리

##### (1-1)데이터 전처리 설정

In [8]:
# 참고 : 주석은 GPT에게 생성시킴.

# 사용한 데이터 전처리 설명:
# 1. transforms.ToTensor() : 데이터를 텐서로 변환. 이미지를 [0, 255] 범위에서 [0.0, 1.0] 범위의 텐서로 스케일링.
# 2. transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) : 변환한 텐서를 정규화. 각 채널에 대해 평균을 0.5로, 표준편차를 0.5로 설정해 데이터의 범위를 [-1, 1]로 조정.

# 훈련용 transforms 생성
# Compose를 사용하여 여러 변환을 연속적으로 적용.
# ToTensor로 이미지를 텐서로 변환 후, Normalize로 채널별로 이미지를 정규화.
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 테스트용 transforms 생성
# 훈련 데이터와 동일한 방식으로 테스트 데이터도 전처리.
# 이는 모델이 훈련 시 사용한 데이터의 분포와 유사하게 테스트 데이터를 조정하기 위함.
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


##### (1-2)데이터셋 및 DataLoader 준비

In [9]:
# 참고 : 주석은 GPT에게 생성시킴.

# CIFAR10 데이터 다운로드 및 transforms 적용 설명
# CIFAR10: torchvision 라이브러리에 포함된 예제 데이터셋. 10개의 다른 클래스(비행기, 자동차, 새 등)의 32x32 컬러 이미지 포함.

# 훈련 데이터셋 다운로드 및 전처리
# root: 데이터 저장 경로, train: True로 설정해 훈련 데이터셋 지정, download: True로 데이터셋이 없을 경우 다운로드
# transform: 정의된 transform_train 적용하여 데이터 전처리
training_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)

# 검증(테스트) 데이터셋 다운로드 및 전처리
# train을 False로 설정해 검증 데이터셋 지정, 나머지는 훈련 데이터셋 설정과 동일
validation_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# DataLoader 객체 생성: 훈련 및 검증 데이터셋에 대한 배치 파이프라인 생성
# batch_size: 한 번에 로드할 데이터의 수, shuffle: 데이터를 섞을지 여부(훈련 데이터는 True로 설정해 매 에포크마다 데이터 섞음)

# 훈련 데이터용 DataLoader
# shuffle=True로 설정해 데이터셋을 무작위로 섞어 모델이 일반화 잘 되도록 함
training_loader = torch.utils.data.DataLoader(training_dataset, batch_size=100, shuffle=True)

# 검증 데이터용 DataLoader
# 검증 데이터는 섞을 필요 없으므로 shuffle=False로 설정
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=100, shuffle=False)

# CIFAR10 클래스 목록
# 실제 데이터는 숫자로 표현되지만, 이해를 돕기 위해 클래스 이름으로 맵핑
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


100%|██████████| 170498071/170498071 [00:02<00:00, 81162028.26it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


#### (2) 모델아키텍쳐 생성

In [13]:
# 참고 : 주석은 GPT에게 생성시킴.

import torch.nn as nn
import torch.nn.functional as F

# 신경망 모델 정의
# 정말 간단한 FFNN(Feedforward Neural Network, 순방향 신경망) 모델
class ANN(nn.Module):
    def __init__(self):
        super(ANN, self).__init__()
        # 입력층에서 첫 번째 은닉층으로의 연결 정의.
        # 입력 차원은 3색상 채널(RGB)을 가진 32x32 이미지, 즉 3*32*32=3072.
        # 첫 번째 은닉층의 노드 수는 120.
        self.fc1 = nn.Linear(3 * 32 * 32, 120)  # 입력 차원: 3색상 채널 x 32 x 32 이미지, 은닉 차원: 120

        # 은닉층의 차원 수, 은닉층 수 등 네트워크 아키텍쳐 자체도 하이퍼파라미터로, 선택 및 조정 필요.
        # 현재는 특별한 이유 없이 120으로 설정.
        self.fc2 = nn.Linear(120, 120)  # 두 번째 은닉층
        self.fc3 = nn.Linear(120, 120)  # 세 번째 은닉층
        self.fc4 = nn.Linear(120, 84)   # 네 번째 은닉층, 노드 수를 84로 줄임(이것도 하나의 하이퍼파라미터)
        self.fc5 = nn.Linear(84, 10)    # 출력층, 10개의 출력 클래스

    def forward(self, x):
        # 이미지를 평탄화하여 1차원 벡터로 변환.
        # 이 과정은 신경망에 이미지 데이터를 적절히 입력하기 위해 필요.
        x = x.view(-1, 3 * 32 * 32)  # 이미지 평탄화
        # 각 은닉층을 거치면서 ReLU 활성화 함수 적용.
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        # 마지막 층에서는 활성화 함수 적용하지 않음. 분류 문제에서 이 출력을 소프트맥스에 전달.
        x = self.fc5(x)
        return x


#### (3) 훈련 및 검증 함수 정의

In [11]:
# 참고 : 주석 및 코드 정리 및 수정을 GPT에게 생성시킴.

def train(training_loader, network, optimizer, loss_fn, device):
    network.train()  # 모델을 훈련 모드로 설정. 이 단계에서는 모델이 학습 가능한 상태가 됨.
    total_loss = 0  # 총 손실을 저장하기 위한 변수 초기화
    correct = 0  # 정확히 예측된 데이터의 수를 저장하기 위한 변수 초기화
    total = 0  # 처리된 총 데이터 수를 저장하기 위한 변수 초기화
    for data, target in training_loader:  # 훈련 데이터 로더를 반복 처리
        data, target = data.to(device), target.to(device)  # 데이터와 타깃을 계산 장치로 이동
        optimizer.zero_grad()  # 이전 스텝의 기울기를 초기화
        output = network(data)  # 네트워크를 통해 입력 데이터에 대한 예측값을 계산 (순전파)
        loss = loss_fn(output, target)  # 예측값과 실제값 사이의 손실을 계산
        loss.backward()  # 손실에 대해 역전파 수행, 각 매개변수의 기울기 계산
        optimizer.step()  # 계산된 기울기를 이용해 매개변수 업데이트

        total_loss += loss.item()  # 현재 배치의 손실을 총 손실에 누적
        _, predicted = torch.max(output, 1)  # 예측값 중 가장 높은 값을 가진 인덱스를 선택
        total += target.size(0)  # 처리된 총 데이터 수 업데이트
        correct += (predicted == target).sum().item()  # 정확히 예측된 데이터의 수 업데이트

    avg_loss = total_loss / len(training_loader)  # 평균 손실 계산
    accuracy = 100 * correct / total  # 정확도 계산
    print(f'Training Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%')  # 평균 손실과 정확도 출력

def test(validation_loader, network, loss_fn, device):
    network.eval()  # 모델을 평가 모드로 설정. 이 단계에서는 모델이 기울기를 계산하지 않음.
    total_loss = 0  # 총 손실을 저장하기 위한 변수 초기화
    correct = 0  # 정확히 예측된 데이터의 수를 저장하기 위한 변수 초기화
    total = 0  # 처리된 총 데이터 수를 저장하기 위한 변수 초기화
    with torch.no_grad():  # 기울기 계산을 비활성화하여 메모리 사용량 줄이고 계산 속도 향상
        for data, target in validation_loader:  # 검증 데이터 로더를 반복 처리
            data, target = data.to(device), target.to(device)
            output = network(data)  # 네트워크를 통해 입력 데이터에 대한 예측값을 계산
            loss = loss_fn(output, target)  # 예측값과 실제값 사이의 손실을 계산
            total_loss += loss.item()  # 현재 배치의 손실을 총 손실에 누적
            _, predicted = torch.max(output, 1)  # 예측값 중 가장 높은 값을 가진 인덱스를 선택
            total += target.size(0)  # 처리된 총 데이터 수 업데이트
            correct += (predicted == target).sum().item()  # 정확히 예측된 데이터의 수 업데이트

    avg_loss = total_loss / len(validation_loader)  # 평균 손실 계산
    accuracy = 100 * correct / total  # 정확도 계산
    print(f'Test Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%')  # 평균 손실과 정확도 출력

    return accuracy  # 계산된 정확도 반환


#### (4)훈련 및 테스트

* 에폭(Epoch), 반복실행(itteration), 배치사이즈(batch)의 경우

In [12]:
# 참고 : 주석은 GPT에게 생성시킴.
# 에폭 수 설정
epochs = 10

# 모델 인스턴스 생성 및 CUDA GPU 사용 설정 (빠른 연산을 위함)
model = ANN().to(device)

# 손실 함수 설정. CrossEntropyLoss는 다중 클래스 분류 문제에서 널리 사용됨.
# Keras 모델에서 사용하는 categorical_crossentropy 손실 함수와 동일한 역할을 함.
loss_fn = nn.CrossEntropyLoss()

# 학습률 설정. 이는 하이퍼파라미터로, 모델 학습 시 학습 속도를 결정함.
learning_rate = 0.001

# 최적화 알고리즘으로 Adam 사용. 모델의 매개변수와 학습률을 인자로 받음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 에폭마다 훈련 및 검증 반복
for i in range(epochs):
    print(f"Epoch {i+1} \n------------------------")
    # 훈련 함수 호출. 훈련 데이터 로더, 모델, 최적화 알고리즘, 손실 함수를 인자로 넘김.
    train(training_loader, model, optimizer, loss_fn, device)
    # 검증 함수 호출. 검증 데이터 로더, 모델, 손실 함수를 인자로 넘김.
    test(validation_loader, model, loss_fn, device)
print("Done!")  # 모든 에폭의 훈련 및 검증이 끝나면 완료 메시지 출력


Epoch 1 
------------------------
Training Loss: 1.6915, Accuracy: 39.43%
Test Loss: 1.5404, Accuracy: 45.40%
Epoch 2 
------------------------
Training Loss: 1.4649, Accuracy: 48.17%
Test Loss: 1.4387, Accuracy: 49.54%
Epoch 3 
------------------------
Training Loss: 1.3603, Accuracy: 52.05%
Test Loss: 1.3855, Accuracy: 51.12%
Epoch 4 
------------------------
Training Loss: 1.2764, Accuracy: 54.89%
Test Loss: 1.3632, Accuracy: 51.86%
Epoch 5 
------------------------
Training Loss: 1.2105, Accuracy: 57.25%
Test Loss: 1.3575, Accuracy: 52.60%
Epoch 6 
------------------------
Training Loss: 1.1548, Accuracy: 59.01%
Test Loss: 1.3440, Accuracy: 53.47%
Epoch 7 
------------------------
Training Loss: 1.1001, Accuracy: 60.86%
Test Loss: 1.3725, Accuracy: 53.28%
Epoch 8 
------------------------
Training Loss: 1.0531, Accuracy: 62.50%
Test Loss: 1.3516, Accuracy: 53.43%
Epoch 9 
------------------------
Training Loss: 1.0104, Accuracy: 63.97%
Test Loss: 1.4063, Accuracy: 52.71%
Epoch 10 


### 인용 및 참고문헌 출처

[1] 파이토치 딥러닝 마스터(2022), 엘리 스티븐스, 루카 안티가, 토마스 피이만 저/현동석 역,책만 p42 및 그림1.1 인용.

[2] Deep Learning(2016), Ian Goodfellow and Yoshua Bengio and Aaron Courville, An MIT Press book (https://www.deeplearningbook.org), 164page 인용

In [ ]:
#<EOD>